In [49]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


In [50]:
#get datasets
household_data = pd.read_csv("NHST_Data\hhpub.csv")
person_data = pd.read_csv("NHST_Data\perpub.csv")
trip_data = pd.read_csv("NHST_Data\\trippub.csv")
vehicle_data = pd.read_csv("NHST_Data\\vehpub.csv")
gas_data = pd.read_csv("data_with_gas_usage.csv")

#use data with vehicles that only use gasoline 
#vehicle_data = vehicle_data.loc[vehicle_data['VEHFUEL'].between(1,2)]
gas_data = gas_data.loc[gas_data['VEHFUEL'] == 1]
gas_data = gas_data.loc[gas_data['GASUSGE'].notna()]

In [51]:
#set up new dataset including unique id from vehicle data
df = pd.DataFrame({'HOUSEID' : pd.unique(gas_data['HOUSEID']),
                   'GASUSGE' : 0,
                   'HighestEDUC' : 0, 'MEDCOND':2,
                   
                   'TAXISERVICE' : 0, 'RIDESHARE22' : 0, 'ESCOOTERUSED' : 0,
                   'PTUSED' : 0, 'MCTRANSIT' : 0, 'WALKTRANSIT' : 0, 
                   'BIKETRANSIT' : 0, 'EMPLOYMENT2' : 0,
                  })

person_col = ['TAXISERVICE', 'RIDESHARE22', 'ESCOOTERUSED', 'PTUSED', 'MCTRANSIT', 'WALKTRANSIT', 'BIKETRANSIT', 'EMPLOYMENT2']

#add empty columns from household data
for col_name in household_data.columns:
    if col_name != 'HOUSEID':
        df[col_name] = 0
        
for i in df.index:
    #get total gasuage of each household
    id = gas_data.loc[gas_data['HOUSEID'] == df['HOUSEID'][i]].index
    if(not id.empty):
        df.loc[i, 'GASUSGE'] = gas_data['GASUSGE'][id].sum()
    
    #get all column values from household data
    id = household_data.loc[household_data['HOUSEID'] == df['HOUSEID'][i]].index[0]
    for col_name in household_data.columns:
        if col_name != 'HOUSEID':
            df.loc[i, col_name] = household_data[col_name][id]
    
         
    id = person_data.loc[person_data['HOUSEID'] == df['HOUSEID'][i]].index
    df.loc[i, 'HighestEDUC'] = max(df.loc[i, 'HighestEDUC'], person_data['EDUC'][id].max())         #get highest education level of each household
    
    for col in person_col:
        #get the number of days where certain alternative travel methods are used by the household in a month
        df.loc[i, col] = max(0, person_data.iloc[id].loc[person_data[col] >= 0][col].sum())


""" 
for i in df.index: 
    df.loc[df['HOUSEID'][i] == 
"""
#df

C:\Users\Jianwei\AppData\Local\Temp\ipykernel_12572\2694761524.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '408.25815788742875' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'GASUSGE'] = gas_data['GASUSGE'][id].sum()
C:\Users\Jianwei\AppData\Local\Temp\ipykernel_12572\2694761524.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4683.5625987' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, col_name] = household_data[col_name][id]
C:\Users\Jianwei\AppData\Local\Temp\ipykernel_12572\2694761524.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '14269.464209' has dtype incompatible with int64, please explicitly cast to a compatible

" \nfor i in df.index: \n    df.loc[df['HOUSEID'][i] == \n"

In [60]:
df

,HOUSEID,GASUSGE,HighestEDUC,MEDCOND,TAXISERVICE,RIDESHARE22,ESCOOTERUSED,PTUSED,MCTRANSIT,WALKTRANSIT,...,URBAN,URBANSIZE,URBRUR,PPT517,YOUNGCHILD,RESP_CNT,URBRUR_2010,TDAYDATE,WRKCOUNT,STRATUMID
0,9000013002,-0.270053,7,2,-0.146325,-0.291888,-0.083774,-0.220185,-0.139403,0.460058,...,1,4,1,2.092350,-0.272258,1.576994,1,202202,-0.031096,1021
1,9000013016,-0.295280,6,2,-0.146325,-0.291888,-0.083774,-0.220185,-0.139403,-0.013900,...,1,2,1,-0.428248,-0.272258,-0.181876,1,202202,1.066596,1021
2,9000013039,-0.386259,6,2,-0.146325,-0.291888,-0.083774,-0.220185,-0.139403,0.065093,...,1,4,1,2.092350,-0.272258,1.576994,1,202201,-0.031096,1021
3,9000013041,-0.385410,6,2,-0.146325,-0.291888,-0.083774,-0.220185,-0.139403,-0.566852,...,1,3,1,-0.428248,-0.272258,-1.061311,1,202201,-0.031096,1021
4,9000013042,0.714016,8,2,-0.146325,0.745818,-0.083774,1.567999,-0.139403,0.144086,...,1,4,1,-0.428248,2.294851,-0.181876,1,202202,1.066596,1021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7161,9000218006,-0.286553,7,2,-0.146325,-0.291888,-0.083774,-0.220185,-0.139403,-0.566852,...,1,3,1,2.092350,-0.272258,0.697559,1,202212,-0.031096,1031
7162,9000218013,-0.420106,7,2,-0.146325,-0.291888,-0.083774,-0.220185,-0.139403,-0.329873,...,1,1,1,-0.428248,-0.272258,-0.181876,1,202301,-1.128787,1031
7163,9000218021,-0.263616,6,2,-0.146325,-0.291888,-0.083774,-0.220185,-0.139403,-0.566852,...,4,6,2,-0.428248,-0.272258,-1.061311,2,202212,-1.128787,1032
7164,9000218034,-0.233573,2,2,-0.146325,-0.291888,-0.083774,-0.220185,-0.139403,-0.566852,...,4,6,2,-0.428248,-0.272258,-0.181876,2,202212,-1.128787,1032


In [53]:
#search for difference of columns between two dataset
#list(set(person_data.columns) - set(household_data.columns))

In [65]:
df.isna().sum()


HOUSEID         0
GASUSGE         0
HighestEDUC     0
MEDCOND         0
TAXISERVICE     0
RIDESHARE22     0
ESCOOTERUSED    0
PTUSED          0
MCTRANSIT       0
WALKTRANSIT     0
BIKETRANSIT     0
EMPLOYMENT2     0
WTHHFIN         0
WTHHFIN5D       0
WTHHFIN2D       0
NUMADLT         0
HOMEOWN         0
HOMETYPE        0
RAIL            0
CENSUS_D        0
CENSUS_R        0
HH_HISP         0
DRVRCNT         0
CNTTDHH         0
CDIVMSAR        0
FLAG100         0
HHFAMINC        0
HHFAMINC_IMP    0
HH_RACE         0
HHSIZE          0
HHVEHCNT        0
HHRELATD        0
LIF_CYC         0
MSACAT          0
MSASIZE         0
TRAVDAY         0
URBAN           0
URBANSIZE       0
URBRUR          0
PPT517          0
YOUNGCHILD      0
RESP_CNT        0
URBRUR_2010     0
TDAYDATE        0
WRKCOUNT        0
STRATUMID       0
dtype: int64

In [55]:
#print(df[df['ANNMILES'] >= df.ANNMILES.quantile(0.9)]['ANNMILES'].values.sum()/df['ANNMILES'].values.sum())
#print(df[df['GASUSGE'] >= df.GASUSGE.quantile(0.9)]['GASUSGE'].values.sum()/df['GASUSGE'].values.sum())

In [56]:
#person_data['CNTTDTR'].value_counts()

In [57]:
#print(household_data.columns)
#print(person_data.columns)
#print(trip_data.columns)
#print(vehicle_data.columns)


In [66]:
#standard scaling all numeric values
num_col = person_col + ["GASUSGE", "WTHHFIN", "WTHHFIN5D",
                        "WTHHFIN2D", "NUMADLT", 'DRVRCNT',
                        'CNTTDHH', 'HHSIZE', 'HHVEHCNT',
                        'PPT517', 'YOUNGCHILD', 'RESP_CNT',
                        'WRKCOUNT',
                        ]

standard_scaler = StandardScaler()
for col in num_col:
    df[col] = standard_scaler.fit_transform(df[[col]])
df

,HOUSEID,GASUSGE,HighestEDUC,MEDCOND,TAXISERVICE,RIDESHARE22,ESCOOTERUSED,PTUSED,MCTRANSIT,WALKTRANSIT,...,URBAN,URBANSIZE,URBRUR,PPT517,YOUNGCHILD,RESP_CNT,URBRUR_2010,TDAYDATE,WRKCOUNT,STRATUMID
0,9000013002,-0.270053,7,2,-0.146325,-0.291888,-0.083774,-0.220185,-0.139403,0.460058,...,1,4,1,2.092350,-0.272258,1.576994,1,202202,-0.031096,1021
1,9000013016,-0.295280,6,2,-0.146325,-0.291888,-0.083774,-0.220185,-0.139403,-0.013900,...,1,2,1,-0.428248,-0.272258,-0.181876,1,202202,1.066596,1021
2,9000013039,-0.386259,6,2,-0.146325,-0.291888,-0.083774,-0.220185,-0.139403,0.065093,...,1,4,1,2.092350,-0.272258,1.576994,1,202201,-0.031096,1021
3,9000013041,-0.385410,6,2,-0.146325,-0.291888,-0.083774,-0.220185,-0.139403,-0.566852,...,1,3,1,-0.428248,-0.272258,-1.061311,1,202201,-0.031096,1021
4,9000013042,0.714016,8,2,-0.146325,0.745818,-0.083774,1.567999,-0.139403,0.144086,...,1,4,1,-0.428248,2.294851,-0.181876,1,202202,1.066596,1021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7161,9000218006,-0.286553,7,2,-0.146325,-0.291888,-0.083774,-0.220185,-0.139403,-0.566852,...,1,3,1,2.092350,-0.272258,0.697559,1,202212,-0.031096,1031
7162,9000218013,-0.420106,7,2,-0.146325,-0.291888,-0.083774,-0.220185,-0.139403,-0.329873,...,1,1,1,-0.428248,-0.272258,-0.181876,1,202301,-1.128787,1031
7163,9000218021,-0.263616,6,2,-0.146325,-0.291888,-0.083774,-0.220185,-0.139403,-0.566852,...,4,6,2,-0.428248,-0.272258,-1.061311,2,202212,-1.128787,1032
7164,9000218034,-0.233573,2,2,-0.146325,-0.291888,-0.083774,-0.220185,-0.139403,-0.566852,...,4,6,2,-0.428248,-0.272258,-0.181876,2,202212,-1.128787,1032


In [67]:
df.to_csv("processed_data.csv", index=False)